# Single Agent Demo (Python)

This notebook demonstrates how to interact with a **single agent** using the Python Agent Framework backend API.

## What You'll Learn
- How to get available agents
- How to send messages to a single agent
- How to handle responses
- Different response formats (user-friendly vs detailed)

## Prerequisites
Make sure the backend API is running on `http://localhost:8000`

```bash
cd Backend/python
python -m uvicorn main:app --reload
```

## Setup - Install Required Packages

In [ ]:
# Install required packages
!pip install requests python-dotenv -q

import requests
import json
from datetime import datetime
from typing import Dict, Any, Optional, List
import time

print("? Packages installed and modules imported successfully!")

## Configure API Connection

In [ ]:
# API Configuration
API_BASE_URL = "http://localhost:8000"
TIMEOUT = 120  # 2 minutes

# Create session for connection pooling
session = requests.Session()

print(f"? Connected to API: {API_BASE_URL}")

## Test API Connection

Let's first verify that the backend API is running and accessible.

In [ ]:
# Quick test to verify API is accessible
try:
    response = session.get(f"{API_BASE_URL}/health", timeout=5)
    print(f"? API Status: {response.status_code}")
    
    if response.status_code == 200:
        print("? Backend API is running and accessible!")
        health_data = response.json()
        print(f"   Status: {health_data.get('status')}")
    else:
        print(f"??  API returned status: {response.status_code}")
        print("Make sure the backend is running with: python -m uvicorn main:app --reload")
except requests.exceptions.RequestException as ex:
    print(f"? Cannot connect to API: {ex}")
    print("Please start the backend API first:")
    print("   cd Backend/python")
    print("   python -m uvicorn main:app --reload")

## Step 1: Get Available Agents

In [ ]:
# Get all available agents
response = session.get(f"{API_BASE_URL}/agents")
agents_data = response.json()

print("?? Available Agents:")
print("="*50)

for agent in agents_data["agents"]:
    name = agent["name"]
    agent_type = agent["type"]
    provider = agent.get("provider", "N/A")
    
    print(f"\n?? Agent: {name}")
    print(f"   Type: {agent_type}")
    print(f"   Provider: {provider}")
    
    if "capabilities" in agent and agent["capabilities"]:
        capabilities = ", ".join(agent["capabilities"])
        print(f"   Capabilities: {capabilities}")

print(f"\n? Total available agents: {agents_data['total']}")

## Step 2: Send a Simple Message to a Single Agent (User-Friendly Format)

In [ ]:
# Prepare chat request for a single agent
chat_request = {
    "message": "What is artificial intelligence and how is it used in healthcare?",
    "agents": ["generic_agent"],  # Single agent
    "session_id": None,  # New session
    "format": "user_friendly"  # Default format
}

print("?? Sending message to single agent...")
print(f"Message: {chat_request['message']}")
print(f"Agent: {chat_request['agents'][0]}\n")

# Send the request
response = session.post(
    f"{API_BASE_URL}/chat",
    json=chat_request,
    timeout=TIMEOUT
)
response.raise_for_status()
result = response.json()

# Display response
print("\n?? Response:")
print("="*50)
print(f"Agent: {result['agent']}")
print(f"Session ID: {result['session_id']}")
print(f"\nContent:\n{result['content']}")

# Display metadata
if "metadata" in result and result["metadata"]:
    metadata = result["metadata"]
    print(f"\n?? Metadata:")
    print(f"   Format: {metadata.get('response_type', 'N/A')}")
    print(f"   Agent Count: {metadata.get('agent_count', 'N/A')}")
    print(f"   Total Turns: {metadata.get('total_turns', 'N/A')}")

## Step 3: Send a Message with Detailed Format

The detailed format provides more information including individual agent responses, turn information, and metadata.

In [ ]:
# Request with detailed format
detailed_request = {
    "message": "Explain machine learning in simple terms.",
    "agents": ["generic_agent"],
    "format": "detailed",  # Detailed format for more information
    "max_turns": 2
}

print("?? Sending message with DETAILED format...")
print(f"Message: {detailed_request['message']}\n")

response = session.post(
    f"{API_BASE_URL}/chat",
    json=detailed_request,
    timeout=TIMEOUT
)
response.raise_for_status()
detailed_result = response.json()

# Display detailed response
print("\n?? Detailed Response:")
print("="*50)
print(f"Conversation ID: {detailed_result['conversation_id']}")
print(f"Total Turns: {detailed_result['total_turns']}")

# Show individual responses
responses = detailed_result["responses"]
print(f"\n?? Individual Agent Responses ({len(responses)}):")

for resp in responses:
    print(f"\n  ?? Agent: {resp['agent']}")
    print(f"     Turn: {resp['metadata']['turn']}")
    print(f"     Content: {resp['content']}")
    print(f"     Timestamp: {resp['metadata']['timestamp']}")

# Summary
if "summary" in detailed_result and detailed_result["summary"]:
    print(f"\n?? Summary:\n{detailed_result['summary']}")

# Metadata
metadata = detailed_result["metadata"]
print(f"\n?? Metadata:")
print(f"   Response Type: {metadata.get('response_type')}")
print(f"   Group Chat Type: {metadata.get('group_chat_type')}")
print(f"   Agent Count: {metadata.get('agent_count')}")

## Step 4: Auto-select Agent (No Agent Specified)

If you don't specify an agent, the system will automatically select one for you.

In [ ]:
# Request without specifying an agent
auto_request = {
    "message": "Tell me a fun fact about space.",
    # No agents specified - will be auto-selected
    "format": "user_friendly"
}

print("?? Sending message with AUTO-SELECTED agent...")
print(f"Message: {auto_request['message']}\n")

response = session.post(
    f"{API_BASE_URL}/chat",
    json=auto_request,
    timeout=TIMEOUT
)
response.raise_for_status()
auto_result = response.json()

print("\n?? Response:")
print("="*50)
print(f"Auto-selected Agent: {auto_result['agent']}")
print(f"\nContent:\n{auto_result['content']}")

## Step 5: Using Long-Running Memory (Optional)

Enable memory to have the agent remember your name and persona across messages.

In [ ]:
# Enable memory to personalize responses
print("?? Testing Long-Running Memory")
print("="*50 + "\n")

# First message - introduce yourself
memory_msg1 = {
    "message": "Hi! My name is Alex and I'm a software developer.",
    "agents": ["generic_agent"],
    "enable_memory": True,  # Enable memory!
    "format": "user_friendly"
}

print(f"[Turn 1] ?? {memory_msg1['message']}")
print(f"         Memory: ON ??\n")

response = session.post(
    f"{API_BASE_URL}/chat",
    json=memory_msg1,
    timeout=TIMEOUT
)
mem_result1 = response.json()
mem_session_id = mem_result1["session_id"]

print(f"?? {mem_result1['content']}\n")
print(f"? Session ID: {mem_session_id}")
print(f"? Memory extracted: Name='Alex', Persona='software developer'\n")

time.sleep(1)

# Follow-up - agent should remember
memory_msg2 = {
    "message": "What's my name and what do I do?",
    "session_id": mem_session_id,
    "agents": ["generic_agent"],
    "enable_memory": True,
    "format": "user_friendly"
}

print(f"[Turn 2] ?? {memory_msg2['message']}")
print(f"         Memory: ON ??\n")

response = session.post(
    f"{API_BASE_URL}/chat",
    json=memory_msg2,
    timeout=TIMEOUT
)
mem_result2 = response.json()

print(f"?? {mem_result2['content']}\n")
print(f"? The agent remembered your information!")
print(f"\n?? Learn more in the 04-LongRunningMemory-Demo.ipynb notebook")

## Step 6: Working with Different Agent Types

Try sending messages to different types of agents to see how they respond differently.

In [ ]:
# Try different agents based on what's available
agent_types = ["generic_agent", "people_lookup", "knowledge_finder"]

for agent_type in agent_types:
    try:
        print(f"\n?? Testing agent: {agent_type}")
        print("="*50)
        
        test_request = {
            "message": "Hello! What can you help me with?",
            "agents": [agent_type],
            "format": "user_friendly"
        }
        
        response = session.post(
            f"{API_BASE_URL}/chat",
            json=test_request,
            timeout=TIMEOUT
        )
        
        if response.status_code == 200:
            test_result = response.json()
            content = test_result["content"]
            truncated_content = content[:200] + "..." if len(content) > 200 else content
            
            print(f"? Agent responded: {truncated_content}")
        else:
            print(f"??  Agent not available or error occurred")
    except Exception as ex:
        print(f"? Error testing {agent_type}: {ex}")

## Summary

In this notebook, you learned:
- ? How to retrieve available agents from the API
- ? How to send messages to a single agent
- ? The difference between user-friendly and detailed response formats
- ? How to enable long-running memory with `enable_memory` flag
- ? How to use auto-agent selection
- ? How to work with different agent types

## Key Features

### Response Formats
- **user_friendly**: Clean, synthesized response (recommended for most cases)
- **detailed**: Full conversation history with metadata (useful for debugging)

### Memory Feature (NEW)
- Enable with `enable_memory: True` in request
- Agent extracts and remembers your name and persona
- Personalizes future responses based on remembered information
- See **04-LongRunningMemory-Demo.ipynb** for complete examples

## Next Steps
- Try the **02-MultipleAgents-Demo** notebook to see how agents collaborate
- Explore the **03-ContentSafety-Demo** for content moderation features
- Check out the **04-LongRunningMemory-Demo** for session management and memory features